In [1]:
from numpy import *; from numpy.core.records import fromarrays;from scipy.constants import *
from matplotlib.pyplot import *
import sys; sys.path.insert(0,'/home/nbirge/Desktop/large_analysis/ca_analysis/'); import os
import predefined as pd; import fileread as fr
from random import seed,shuffle


def precuts(x,twindow=250,pilewindow=100,energy=100,remove_double=False,\
            event_window=250,keep_pulser=False,oscillation=False,osc_amp=50):
    '''Returns portion of data with > twindow between separate neighboring events,
        <pilewindow for detected pileups, and have an energy > energy'''
    types=x.dtype
    types=types.fields.keys()
    
    if remove_double==True:
        x=x[doubles(x)]
        
    
    x=x[x['energy']>energy]
    x.sort(order='timestamp')
    
    if oscillation == True and 'osc_amps' in types:
        t = np.sqrt(np.sum(x['osc_amps'][:,0:2]**2.,axis=1)) < osc_amp
        x=x[t]
        
    t0,t1,t2=x['timestamp'][0:-2],x['timestamp'][1:-1],x['timestamp'][2:]
    trutharray=(t2-t1>event_window)*(t1-t0>event_window)
    if keep_pulser==True:
        trutharray+=x['energy'][1:-1]>2500
    x=x[1:-1][trutharray]
    x=x[x['pileup']<2+(np.abs(x['pilediff'])<pilewindow)]
    x=x[np.abs(x['pilediff']+10000)>100]

    return x

# Make the oscillation cut!!

In [5]:
fs=20
path='/run/media/nbirge/45Ca_data/cur_data/'
runs=[run for run in os.listdir(path) if run.endswith('-all.npy')]
badruns=[run for run in os.listdir(path+'bad_runs/') if run.endswith('-all.npy')]
j=41
# seed(10)
# shuffle(runs)
runs=runs+badruns
runs.sort()

for run in runs:
    print('\r'+50*' ',end='')
    print('\r Working on run: '+run[:-4],end='' )
    if run in badruns:
        data=load(path+'bad_runs/'+run)
    else:
        data=load(path+run)
    data=data.view(recarray)
    data.sort(order='timestamp')
    data=data[0:-500]

    time=data.timestamp[-1]-data.timestamp[0]
    fig,ax=subplots(nrows=1,figsize=(15,10),num=1)

    nbins,rang=2000,[0,5000]
    bdchs=np.arange(48)
    for bdch in bdchs:
        bd,ch=divmod(bdch,8)
        if bdch>48 or pd.pixel(bd,ch)=='N/A' or pd.pixel(bd,ch)=='Psr' or not(pd.pixel(bd,ch) in ['64E','64W','65W']):
            continue

        t=(data.board==bd)*(data.channel==ch)

        d_hist,bins=np.histogram(data.energy[t],bins=nbins,range=rang)
        bins=pd.cbins(bins)

        if np.sum(d_hist)>1000:
            ax.plot(bins,d_hist,ds='steps',label=pd.pixel(bd,ch)+' {:0.1e}'.format(np.sum(d_hist)))
        else:
            ax.plot(bins,d_hist,ds='steps')

    #     osc_amp=30
    #     t *= np.sqrt(np.sum(data.osc_amps[:,0:2]**2.,axis=1)) < osc_amp  #OSCILLATION CUT
    #     d_hist,bins=np.histogram(data.energy[t],bins=nbins,range=rang)
    #     bins=pd.cbins(bins)
    #     ax[1].plot(bins,d_hist,ds='steps',label=pd.pixel(bd,ch))

    ax.set_xlim(0,1800)
    ax.set_ylim(10,2e4)
    ax.set_yscale('log')
    ax.legend(fontsize=fs)
    ax.set_title(run[4:-8]+': {:0.1f} mins.'.format(time*4e-9/60),fontsize=fs)
    savefig('./spectra-no_cuts/Run_'+run[4:-8]+'_spectrum')
    close(fig=1)
    
src_path='/run/media/nbirge/45Ca_data/cur_data/calib_src_runs/'
src_runs=[run for run in os.listdir(path) if run.endswith('-all.dat')]
for run in src_runs:    
    data=fr.gen_output(src_path+run)[0].view(np.recarray)
    data.sort(order='timestamp')
    data=data[:-500]
    time=data.timestamp[-1]-data.timestamp[0]
    fig,ax=subplots(nrows=1,figsize=(15,10),num=1)

    nbins,rang=2000,[0,5000]
    bdchs=np.arange(48)
    for bdch in bdchs:
        bd,ch=divmod(bdch,8)
        if bdch>48 or pd.pixel(bd,ch)=='N/A' or pd.pixel(bd,ch)=='Psr' or not(pd.pixel(bd,ch) in ['64E','64W','65W']):
            continue

        t=(data.board==bd)*(data.channel==ch)

        d_hist,bins=np.histogram(data.energy[t],bins=nbins,range=rang)
        bins=pd.cbins(bins)

        if np.sum(d_hist)>1000:
            ax.plot(bins,d_hist,ds='steps',label=pd.pixel(bd,ch)+' {:0.1e}'.format(np.sum(d_hist)))
        else:
            ax.plot(bins,d_hist,ds='steps')

    #     osc_amp=30
    #     t *= np.sqrt(np.sum(data.osc_amps[:,0:2]**2.,axis=1)) < osc_amp  #OSCILLATION CUT
    #     d_hist,bins=np.histogram(data.energy[t],bins=nbins,range=rang)
    #     bins=pd.cbins(bins)
    #     ax[1].plot(bins,d_hist,ds='steps',label=pd.pixel(bd,ch))

    ax.set_xlim(0,1800)
    ax.set_ylim(10,2e4)
    ax.set_yscale('log')
    ax.legend(fontsize=fs)
    ax.set_title(run[4:-8]+': {:0.1f} mins.'.format(time*4e-9/60),fontsize=fs)
    savefig('./spectra-no_cuts/Run_'+run[4:-8]+'_spectrum')
    close(fig=1)


 Working on run: Run_99-all                       